In [ ]:
# imports & paths

import json
import math

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import MinMaxScaler

from statsmodels.tsa.arima.model import ARIMA

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

from pathlib import Path
import sys

# --- Proje kökünü ve src klasörünü bul ---
CURRENT_DIR = Path.cwd()

PROJECT_ROOT = None
SRC_DIR = None

for base in [CURRENT_DIR, *CURRENT_DIR.parents]:
    cand = base / "server" / "src"
    if cand.exists():
        PROJECT_ROOT = base
        SRC_DIR = cand
        break

if SRC_DIR is None:
    raise RuntimeError(f"server/src bulunamadı, CURRENT_DIR = {CURRENT_DIR}")

if str(SRC_DIR) not in sys.path:
    sys.path.insert(0, str(SRC_DIR))

# Çıktı klasörleri
OUTPUTS_DIR = PROJECT_ROOT / "server" / "outputs"
JSON_DIR = OUTPUTS_DIR / "json"

# --- Proje modülleri ---
from data_loading import load_clean_data
from models_baseline import (
    naive_forecast_last_value,
    moving_average_forecast,
    train_test_split_series,
)
from portfolio import build_recommended_portfolios


PROJECT_ROOT: /Users/omerfsaribal/Uni/5th-Term/FinTech/Project/wealthflow/python-server
SRC_DIR: /Users/omerfsaribal/Uni/5th-Term/FinTech/Project/wealthflow/python-server/server/src
OUTPUTS_DIR: /Users/omerfsaribal/Uni/5th-Term/FinTech/Project/wealthflow/python-server/server/outputs
JSON_DIR: /Users/omerfsaribal/Uni/5th-Term/FinTech/Project/wealthflow/python-server/server/outputs/json


In [ ]:
# Cell 3: Load SPY clean data and train/test split

# SPY temiz fiyat serisini yükle
df_spy = load_clean_data("SPY")

# Date index'e al
if "Date" in df_spy.columns:
    df_spy = df_spy.set_index("Date")

df_spy = df_spy.sort_index()
spy_series = df_spy["Close"].astype(float)

print("SPY series length:", len(spy_series))
print("SPY date range:", spy_series.index.min(), "->", spy_series.index.max())

# Time-based split (same idea as models_ml.py / baseline)
SPLIT_DATE = "2023-01-01"  # istersen 2022-12-31 kullan, önemli olan mantık

spy_train, spy_test = train_test_split_series(spy_series, split_date=SPLIT_DATE)

print("Train length:", len(spy_train), "Test length:", len(spy_test))
print("Train last date:", spy_train.index.max(), "Test first date:", spy_test.index.min())

In [ ]:
# Cell 4: Baseline forecasts (naive & moving average) and metrics

horizon = len(spy_test)

# Naive: son değeri test boyunca sabit tahmin
baseline_naive = naive_forecast_last_value(spy_train, horizon=horizon)

# Moving average: son 30 gün ortalamasını sabit tahmin
baseline_ma = moving_average_forecast(spy_train, window=30, horizon=horizon)

y_true = spy_test.values

def compute_metrics(y_true, y_pred, name="model"):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = math.sqrt(mean_squared_error(y_true, y_pred))
    print(f"{name}: MAE={mae:.4f}, RMSE={rmse:.4f}")
    return mae, rmse

mae_naive, rmse_naive = compute_metrics(y_true, baseline_naive, name="Naive baseline")
mae_ma, rmse_ma = compute_metrics(y_true, baseline_ma, name="MovingAverage baseline")

In [ ]:
# Cell 5: ARIMA model on SPY

# Basit bir ARIMA(1,1,1) modeli. İstersen order'ı değiştirip oynayabilirsin.
arima_order = (1, 1, 1)

arima_model = ARIMA(spy_train, order=arima_order)
arima_result = arima_model.fit()

print(arima_result.summary())

# Test horizon kadar ileri tahmin
arima_forecast = arima_result.forecast(steps=horizon)

mae_arima, rmse_arima = compute_metrics(y_true, arima_forecast, name=f"ARIMA{arima_order}")

In [ ]:
# Cell 6: LSTM helpers (series -> supervised sequences, model builder)

def create_lstm_sequences(series: pd.Series, lookback: int = 60):
    """
    series: pandas Series of prices (train kısmı).
    lookback: kaç günlük history ile 1 gün sonrası tahmini yapılacak.
    """
    values = series.values.reshape(-1, 1).astype(float)
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled = scaler.fit_transform(values)

    X, y = [], []
    for i in range(lookback, len(scaled)):
        X.append(scaled[i - lookback:i, 0])
        y.append(scaled[i, 0])

    X = np.array(X)
    y = np.array(y)

    # reshape to (samples, timesteps, features)
    X = X.reshape((X.shape[0], X.shape[1], 1))

    return X, y, scaler


def build_lstm_model(lookback: int = 60):
    model = Sequential()
    model.add(LSTM(32, input_shape=(lookback, 1)))
    model.add(Dense(1))

    model.compile(optimizer="adam", loss="mse")
    return model

In [ ]:
# Cell 7: Train LSTM on SPY and multi-step forecast for test period

LOOKBACK = 60

# Train set üzerinden supervised veri oluştur
X_train, y_train_seq, scaler = create_lstm_sequences(spy_train, lookback=LOOKBACK)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train_seq.shape)

lstm_model = build_lstm_model(lookback=LOOKBACK)

early_stop = EarlyStopping(
    monitor="loss",
    patience=5,
    restore_best_weights=True,
)

history = lstm_model.fit(
    X_train,
    y_train_seq,
    epochs=20,
    batch_size=32,
    verbose=1,
    callbacks=[early_stop],
)

# Multi-step forecast:
#  - Başlangıç penceresi: train'in son LOOKBACK günü
#  - Her adımda 1 gün tahmin et, pencereyi kaydır, scaled space'te çalış, sonra inverse transform ile price'a çevir.

train_values = spy_train.values.reshape(-1, 1).astype(float)
train_scaled = scaler.transform(train_values)
history_scaled = list(train_scaled.flatten())

lstm_forecast = []

for i in range(horizon):
    if len(history_scaled) < LOOKBACK:
        # yeterli history yoksa boş geç
        break

    x_input = np.array(history_scaled[-LOOKBACK:])
    x_input = x_input.reshape((1, LOOKBACK, 1))

    yhat_scaled = lstm_model.predict(x_input, verbose=0)[0, 0]

    # history'ye ekle
    history_scaled.append(yhat_scaled)

    # original price scale'e çevir
    yhat_price = scaler.inverse_transform([[yhat_scaled]])[0, 0]
    lstm_forecast.append(yhat_price)

# LSTM forecast uzunluğu test ile aynı olmalı (aksi halde min len'e göre kırparız)
min_len = min(len(y_true), len(lstm_forecast))
y_true_lstm = y_true[:min_len]
lstm_forecast = np.array(lstm_forecast[:min_len])

mae_lstm, rmse_lstm = compute_metrics(y_true_lstm, lstm_forecast, name="LSTM")

In [ ]:
# Cell 8: Comparison table for MAE and RMSE

metrics_df = pd.DataFrame(
    {
        "MAE": {
            "Naive": mae_naive,
            "MovingAverage": mae_ma,
            "ARIMA": mae_arima,
            "LSTM": mae_lstm,
        },
        "RMSE": {
            "Naive": rmse_naive,
            "MovingAverage": rmse_ma,
            "ARIMA": rmse_arima,
            "LSTM": rmse_lstm,
        },
    }
)

metrics_df

In [ ]:
# Optional: bar plot for RMSE comparison

plt.figure(figsize=(6, 4))
metrics_df["RMSE"].plot(kind="bar")
plt.title("SPY – RMSE comparison (Naive vs MA vs ARIMA vs LSTM)")
plt.ylabel("RMSE")
plt.tight_layout()
plt.show()

In [ ]:
# Cell 9: Real vs predicted prices for SPY (test set)

plt.figure(figsize=(12, 6))
plt.plot(spy_test.index[:len(y_true_lstm)], y_true_lstm, label="Actual (SPY test)")
plt.plot(spy_test.index[:len(baseline_naive)], baseline_naive[:len(y_true_lstm)], label="Naive baseline")
plt.plot(spy_test.index[:len(arima_forecast)], arima_forecast[:len(y_true_lstm)], label="ARIMA")
plt.plot(spy_test.index[:len(lstm_forecast)], lstm_forecast, label="LSTM")
plt.xlabel("Date")
plt.ylabel("Price")
plt.title("SPY – Actual vs Predicted (test period)")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Cell 10: Compare baseline forecasts vs ML-enhanced forecasts

with (JSON_DIR / "forecasts_baseline.json").open("r", encoding="utf-8") as f:
    forecasts_baseline = json.load(f)

with (JSON_DIR / "forecasts.json").open("r", encoding="utf-8") as f:
    forecasts_ml = json.load(f)

tickers = ["AAPL", "MSFT", "SPY", "QQQ", "TLT", "GLD"]

rows = []
for t in tickers:
    base = forecasts_baseline.get(t, {})
    ml = forecasts_ml.get(t, {})

    rows.append(
        {
            "ticker": t,
            "base_ret_30d": base.get("expected_return_30d"),
            "base_vol_30d": base.get("expected_vol_30d"),
            "ml_ret_30d": ml.get("expected_return_30d"),
            "ml_vol_30d": ml.get("expected_vol_30d"),
        }
    )

forecasts_compare_df = pd.DataFrame(rows).set_index("ticker")
forecasts_compare_df

In [ ]:
# Cell 11: Build recommended portfolios (if not already built) and inspect JSON

# Eğer portfolio.py CLI ile zaten çalıştırdıysan, JSON hazırdır.
# Yine de burada bir kez daha build edebiliriz (aynı sonucu üretir).

recs = build_recommended_portfolios()

# JSON'a yaz (idempotent)
portfolio_json_path = JSON_DIR / "portfolio_recommendations_notebook.json"
with portfolio_json_path.open("w", encoding="utf-8") as f:
    json.dump(recs, f, indent=2)

portfolio_json_path, recs

In [ ]:
# Cell 12: Weights table and bar plot for one profile

# recs sözlüğü: profile -> {weights, expected_return, expected_risk}
profiles = list(recs.keys())
profiles

# Weights DataFrame
weights_data = {}
for profile, data in recs.items():
    weights_data[profile] = data["weights"]

weights_df = pd.DataFrame(weights_data)
weights_df

In [ ]:
# Bar plot for balanced profile weights

profile_name = "balanced"
w = weights_df[profile_name]

plt.figure(figsize=(6, 4))
w.plot(kind="bar")
plt.title(f"{profile_name.capitalize()} portfolio weights")
plt.ylabel("Weight")
plt.tight_layout()
plt.show()

In [ ]:
# Expected return and risk per profile

rows = []
for profile, data in recs.items():
    rows.append(
        {
            "profile": profile,
            "expected_return_30d": data["expected_return"],
            "expected_risk_30d": data["expected_risk"],
        }
    )

portfolio_stats_df = pd.DataFrame(rows).set_index("profile")
portfolio_stats_df

In [ ]:
# Risk-return scatter plot

plt.figure(figsize=(6, 4))
plt.scatter(
    portfolio_stats_df["expected_risk_30d"],
    portfolio_stats_df["expected_return_30d"],
)

for profile in portfolio_stats_df.index:
    x = portfolio_stats_df.loc[profile, "expected_risk_30d"]
    y = portfolio_stats_df.loc[profile, "expected_return_30d"]
    plt.annotate(profile, (x, y))

plt.xlabel("Expected 30d Risk")
plt.ylabel("Expected 30d Return")
plt.title("Portfolio profiles – Risk vs Return (30d)")
plt.tight_layout()
plt.show()